In [1]:
# Find out all tokens getting mapped to unk with Marian Tokenizer

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-af-fr")

/home/rana/Projects/zindi/transformers/src/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
from datasets import load_dataset,concatenate_datasets
zindi_ds = load_dataset("uvci/Koumankan_mt_dyu_fr")
def preprocess_function(examples):
    inputs = [example["dyu"] for example in examples["translation"]]
    targets = [example["fr"] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=48, truncation=True, padding="max_length")
    # Check for None values in input_ids and labels
    if None in model_inputs["input_ids"] or None in model_inputs["labels"]:
        print("Warning: None values found in tokenized output")
        # Remove examples with None values
        valid_indices = [i for i, (inp, lab) in enumerate(zip(model_inputs["input_ids"], model_inputs["labels"]))
                         if inp is not None and lab is not None]
        for key in model_inputs.keys():
            model_inputs[key] = [model_inputs[key][i] for i in valid_indices]
    return model_inputs
tokenized_zds = zindi_ds.map(
    preprocess_function,
    batched=True,
    remove_columns=zindi_ds["train"].column_names  # Remove original columns
)
concat_ds = concatenate_datasets([tokenized_zds['train'], tokenized_zds['test']])

Map: 100%|██████████| 1393/1393 [00:00<00:00, 2073.88 examples/s]


In [13]:
%cd ..

/home/rana/Projects/zindi


/home/rana/.cache/pypoetry/virtualenvs/zindi-z3yfXQo9-py3.9/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [18]:
with open('data/all_dyu.txt', 'r') as f:
    dyu_data = [line.strip() for line in f]
with open('data/all_fr.txt', 'r') as f:
    fr_data = [line.strip() for line in f]

all_data = dyu_data+fr_data
len(dyu_data), len(fr_data), len(all_data)

(15563, 15563, 31126)

In [39]:
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-af-fr")

# Your corpus of data (replace this with your actual data)
corpus = all_data

# Function to find tokens mapped to <unk>
def find_unk_tokens(text):
    tokens = tokenizer.tokenize(text)
    ids = tokenizer.convert_tokens_to_ids(tokens)
    unk_tokens = [token for token, id in zip(tokens, ids) if id == 1]
    return unk_tokens

# Process the corpus
all_unk_tokens = set()
for sentence in corpus:
    unk_tokens = find_unk_tokens(sentence)
    all_unk_tokens.update(unk_tokens)

# Print the results
print("Tokens mapped to <unk>:")
for token in all_unk_tokens:
    print(token)

/home/rana/Projects/zindi/transformers/src/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Tokens mapped to <unk>:
ɲ
–
’
»
”
«
ɛɲɛ
ɛɲ
ɲɛɛ
ğ
ɛɛ
—«
υ
—
’«
ɲɛɲɛ
“
Õ
ɲɛ
ŋ
̋
ʻ
Ɔ
ɛ
ɛ”
ễ


In [32]:
with open('data/unmapped_fr_tokens.txt','w') as f:
    for token in all_unk_tokens:
         f.write(token + '\n')

In [21]:
len(all_unk_tokens)

26

In [41]:
tokenizer.add_tokens(list(all_unk_tokens))

26

In [42]:
tokenizer.save_pretrained("tokenizer_custom/marian_v2")

('tokenizer_custom/marian_v2/tokenizer_config.json',
 'tokenizer_custom/marian_v2/special_tokens_map.json',
 'tokenizer_custom/marian_v2/vocab.json',
 'tokenizer_custom/marian_v2/source.spm',
 'tokenizer_custom/marian_v2/target.spm',
 'tokenizer_custom/marian_v2/added_tokens.json')